Get all the issuer:
    ------------------

In [1]:
import ordered_set
with open('../company.csv','r') as f:
    c = f.readlines()
companies =  ordered_set.OrderedSet()
for item in c[1:]:
    companies.add(item.strip('\n", ').split(',')[0])


In [2]:
from code.Matching import fuzzCompany2

In [3]:

syn=[('J.P. MORGAN','JPMORGAN'), ('JPMORGAN','J.P. MORGAN'),('J.P.MORGAN','J.P. MORGAN'), ('aktiengesellschaft','ag'),
    ('Limited liability company','llc'),('Public limited company','plc'),('incorporation','inc'),
    ('Sociedad por Acciones','sa'),('Public Limited','ltd'),('int','international'), ('BISHOPSGATE','london'),
    ('FORT DUP 11046','paris'),('GREAT WINCHE','london')]

fcomp = fuzzCompany2(companies,syn)
#fcomp = fuzzCompany2.from_file('fc_data.pkl')

In [4]:
import cPickle as pickle
#with open('fc_data.pkl', 'wb') as output:
    pickle.dump(fcomp, output)

In [4]:
fcomp.closerMatch('az GOLMAN SACHS sfd er ze ae INTernationl er qd')
#fcomp.to_numberset('az GOLMAN SACHS INTernation er qd')

(0.35, 'GOLDMAN SACHS INTERNATIONAL')

In [6]:

def issuer_line(s):
    
    if re.search(r'.*issuer[\s\n]*:.*', s.lower()) or re.match(r'issuer\s{4,}.*', s.lower()):
        #print s
        value, name = fcomp.closerMatch(s.replace('issuer',''))
        if value > 0.1:
            return value, name

    test=re.findall(r'issued by.*', s.lower())
    if len(test) > 0:
        test=test[0].split('\n')[0].replace('issued by','')
        #print test
        value, name = fcomp.closerMatch(test)
        if value > 0.1:
            return value, name

    s=s.replace('\n',' ')
    s=s.replace('\r',' ')
    test=re.findall(r'.*issue of', s.lower())
    if len(test) > 0:
        #print test
        value, name=test[0].replace('issue of','')
        if value > 0.1:
            return value, name

    test=re.findall(r'.*issued', s.lower())
    if len(test) > 0:
        #print test
        test=test[0].replace('issue of','')
        value, name = fcomp.closerMatch(test)
        if value > 0.1:
            return value, name

def get_issuer(doc):
    choices = []             
    txt=doc.lower().replace('\n',' ').replace('\r','').replace('\t',' ')
    for s in tr.split_text(txt):
        if re.match('[iI]+[sS]+[uU]+[eEiI]+[nNrR]?',s):
            print s
            tmp = issuer_line(s)
            if tmp is not None:
                choices.append(tmp)

    if len(choices) > 0:
        choices = sorted(choices,reverse=True)
        print choices
        return choices[0][1]                          
            
    return None


In [1]:
from code.Matching import fuzzCompany2
import os
from code import Text_reader as tr
from code.Labels import label
from code.Text_reader import htmlDoc
import copy 
import re

data0='../data_0'
data1='../data_1'
labels=label.from_cvs(data0)
data=copy.deepcopy(labels)
data.empty()

fcomp = fuzzCompany2.from_file('fc_data.pkl')

The PythonMagick package is missing, get_text_adv function can not be used 
['fileId', 'NOMINAL.CURR', 'MIN.TRAD.AMT', 'MULT.TRAD.AMT', 'ZERO.COUPN.FLAG', 'SEC.SUB.ID', 'FUNG.FL', 'OpCurrency', 'issuerName', 'issuerCity', 'issuerCountry', 'guarantorName', 'guarantorCity', 'guarantorCountry', 'City.Id', 'City.Name']


In [5]:
print_info=False

def get_issuer(txt):
    
    choices=[]
    for s in tr.split_text(txt):   # split into sentences.  
    #for s in txt.split('\n'):
        
        s = s.lower()
        
        #if 'issuer' in s.lower():
        #print repr(s)
        #    print '-----'
        
        if re.search(r'.*issuer[\s\n]*:.*', s.lower()) or re.match(r'issuer\s{4,}.*', s.lower()):
            #print repr(s.replace('issuer',''))
            #ss = re.findall('issuer(.*?\s){10}',s)[0]
            
            tmp = fcomp.closerMatch(s.replace('issuer',' '))
            if tmp is not None:
                value, name = tmp
                #print '-----'
                if print_info:
                    print name, value, s.replace('issuer','')
                if value > 0.8:
                    return name
                elif value > 0.1:
                    choices.append((value,name))
            
        test=re.findall(r'issued by.*', s.lower())
        if len(test) > 0:
            test=test[0].split('\n')[0].replace('issued by','')
            #print test
            tmp = fcomp.closerMatch(test)
            if tmp is not None:
                value, name = tmp
                if print_info:
                    print name, value, test
                if value > 0.8:
                    return name
                elif value > 0.1:
                    choices.append((value,name))
        
        s=s.replace('\n',' ')
        s=s.replace('\r',' ')
        test=re.findall(r'.*issue of', s.lower())
        if len(test) > 0:
            #print test
            test=test[0].replace('issue of','')
            tmp = fcomp.closerMatch(test)
            if tmp is not None:
                value,name = tmp
                if print_info:
                    print name, value, test
                if value > 0.8:
                    return name
                elif value > 0.1:
                    choices.append((value,name))
            
        test=re.findall(r'.*issued', s.lower())
        if len(test) > 0:
            #print test
            test=test[0].replace('issue','')
            tmp = fcomp.closerMatch(test)
            if tmp is not None:
                value, name = tmp
                if print_info:
                    print name, value, test
                #print name, value, test
                if value > 0.8:
                    return name
                elif value > 0.1:
                    choices.append((value,name))
    
    if len(choices) > 0:
        choices = sorted(choices,reverse=True)
        if print_info:
            print choices
        return choices[0][1]  
    
    return None

In [6]:
folder='../data_0/html'
files=tr.get_files(folder)
filesTxT=tr.get_files('../data_0/txt')

for isin in data: # for each isin in the data object created before
#for isin in ['XS1183228789']:
    f=data[isin][0] # get the file_ids
    issuer=None
    for item in f: # for each of these files
        # get the txt.
        txt=tr.clean_text(tr.get_text(filesTxT[item],raw=True))
        issuer=get_issuer(txt)
        if issuer is None:
            txt=tr.clean_text(tr.get_text(files[item]))
            issuer=get_issuer(txt)
            
        if issuer is not None:
            break
    
    print isin, issuer
    if issuer is not None:
        data[isin][8]=issuer

XS1240015799 CREDIT SUISSE INTERNATIONAL
XS1293137904 BNP PARIBAS ARBITRAGE
XS1313796846 BNP PARIBAS ARBITRAGE
XS1313802677 BNP PARIBAS ARBITRAGE
XS1313800895 BNP PARIBAS ARBITRAGE
XS1267246327 GOLDMAN SACHS INTERNATIONAL
XS1275964507 DEUTSCHE BANK AG
XS1329979261 THE HONGKONG AND SHANGHAI BANKING CORPORATION LIMITED
XS1312160507 ROYAL BANK OF CANADA
XS1253932765 CREDIT SUISSE INTERNATIONAL
XS1267246244 GOLDMAN SACHS INTERNATIONAL
XS1293817877 THE HONGKONG AND SHANGHAI BANKING CORPORATION LIMITED
XS1051370440 DEUTSCHE BANK AG
XS1267221841 GOLDMAN SACHS INTERNATIONAL
XS1266266813 BARCLAYS BANK PLC
XS1313804459 BNP PARIBAS ARBITRAGE
XS1332626016 ROYAL BANK OF CANADA
XS1338871996 SMBC NIKKO SECURITIES INC.
XS1051380837 None
XS1313800119 BNP PARIBAS ARBITRAGE
XS1293807720 THE HONGKONG AND SHANGHAI BANKING CORPORATION LIMITED
XS1326568430 ROYAL BANK OF CANADA
XS1222904416 BNP PARIBAS ARBITRAGE
XS1339630227 ROYAL BANK OF CANADA
XS1267238498 GOLDMAN SACHS INTERNATIONAL
XS1254344952 CREDIT SUI

In [7]:
data.evaluate([8],labels,printerr=True, match_ok=100)

XS1051380837 ,  None ,  DEUTSCHE BANK AG
XS1324906947 ,  None ,  HSBC BANK PLC
XS1190870821 ,  JPMORGAN CHASE BANK ,  J.P. MORGAN STRUCTURED PRODUCT
XS1324899985 ,  None ,  HSBC BANK PLC
XS1289291186 ,  INDUSTRIAL & COMMERCIAL BANK OF CHINA LIMITED ,  CREDIT SUISSE INTERNATIONAL
XS1190881190 ,  JPMORGAN CHASE BANK ,  J.P. MORGAN STRUCTURED PRODUCT
XS1051370010 ,  AMERICAN INTERNATIONAL GROUP ,  DEUTSCHE BANK AG
XS1336933194 ,  None ,  HSBC BANK PLC
XS1190873098 ,  JPMORGAN CHASE BANK ,  J.P. MORGAN STRUCTURED PRODUCT
XS1190837291 ,  JPMORGAN CHASE BANK ,  J.P. MORGAN STRUCTURED PRODUCT
XS1190886314 ,  JPMORGAN CHASE BANK ,  J.P. MORGAN STRUCTURED PRODUCT
XS1190837960 ,  JPMORGAN CHASE BANK ,  J.P. MORGAN STRUCTURED PRODUCT
XS1336567315 ,  None ,  HSBC BANK PLC
XS1190866803 ,  JPMORGAN CHASE BANK ,  J.P. MORGAN STRUCTURED PRODUCT
XS1190837374 ,  JPMORGAN CHASE BANK ,  J.P. MORGAN STRUCTURED PRODUCT
XS1332081352 ,  None ,  HSBC BANK PLC
XS1051372495 ,  None ,  DEUTSCHE BANK AG
XS04613323

In [4]:
data.evaluate([8],labels,printerr=True, match_ok=100) # Spliting on line.

XS1051380837 ,  None ,  DEUTSCHE BANK AG
XS1324906947 ,  None ,  HSBC BANK PLC
XS1190870821 ,  None ,  J.P. MORGAN STRUCTURED PRODUCT
XS1324899985 ,  None ,  HSBC BANK PLC
XS1289291186 ,  INDUSTRIAL & COMMERCIAL BANK OF CHINA LIMITED ,  CREDIT SUISSE INTERNATIONAL
XS1266629473 ,  None ,  SG ISSUER
XS1190881190 ,  None ,  J.P. MORGAN STRUCTURED PRODUCT
XS1051370010 ,  AMERICAN INTERNATIONAL GROUP ,  DEUTSCHE BANK AG
XS1266598736 ,  None ,  SG ISSUER
XS1336933194 ,  None ,  HSBC BANK PLC
XS1266625992 ,  None ,  SG ISSUER
XS1190873098 ,  None ,  J.P. MORGAN STRUCTURED PRODUCT
XS1222904929 ,  None ,  BNP PARIBAS ARBITRAGE ISSUANCE
XS1190837291 ,  None ,  J.P. MORGAN STRUCTURED PRODUCT
XS1266626453 ,  None ,  SG ISSUER
XS1289614965 ,  KINGDOM OF THE NETHERLANDS ,  SG ISSUER
XS1190886314 ,  None ,  J.P. MORGAN STRUCTURED PRODUCT
XS1190837960 ,  None ,  J.P. MORGAN STRUCTURED PRODUCT
XS1336567315 ,  None ,  HSBC BANK PLC
XS1190866803 ,  None ,  J.P. MORGAN STRUCTURED PRODUCT
XS1190837374 ,  N

In [23]:
from multiprocessing import Pool

folder='../data_0/html'
files=tr.get_files(folder)
filesTxT=tr.get_files('../data_0/txt')

def get_issuer_data(f):
    isin, files, filesTxT, files  = f
    issuer=None
    for item in files: # for each of these files
        # get the txt.
        txt=tr.clean_text(tr.get_text(filesTxT[item],raw=True))
        issuer=get_issuer(txt)
        if issuer is None:
            txt=tr.clean_text(tr.get_text(files[item]))
            issuer=get_issuer(txt)
            
        if issuer is not None:
            break     
    return (isin,issuer)

fdata = [(isin , data[isin][0], filesTxT, files) for isin in data]
#p = Pool(4)
#ans = p.map(get_issuer_data, fdata)
print fdata
#for isin, issuer in ans:
#    data[isin][8]=issuer 

XS1240015799 ,  KINGDOM OF THE NETHERLANDS ,  CREDIT SUISSE INTERNATIONAL
XS1293137904 ,  KINGDOM OF THE NETHERLANDS ,  BNP PARIBAS ARBITRAGE ISSUANCE
XS1313796846 ,  KINGDOM OF THE NETHERLANDS ,  BNP PARIBAS ARBITRAGE ISSUANCE
XS1313802677 ,  KINGDOM OF THE NETHERLANDS ,  BNP PARIBAS ARBITRAGE ISSUANCE
XS1231203701 ,  KINGDOM OF THE NETHERLANDS ,  DEUTSCHE BANK AG
XS1313800895 ,  KINGDOM OF THE NETHERLANDS ,  BNP PARIBAS ARBITRAGE ISSUANCE
XS1267246327 ,  KINGDOM OF THE NETHERLANDS ,  GOLDMAN SACHS INTERNATIONAL
XS1275964507 ,  KINGDOM OF THE NETHERLANDS ,  DEUTSCHE BANK AG
XS1329979261 ,  KINGDOM OF THE NETHERLANDS ,  THE HONGKONG AND SHANGHAI BANK
XS1326567895 ,  KINGDOM OF THE NETHERLANDS ,  ROYAL BANK OF CANADA
XS1253932765 ,  KINGDOM OF THE NETHERLANDS ,  CREDIT SUISSE INTERNATIONAL
XS1267246244 ,  KINGDOM OF THE NETHERLANDS ,  GOLDMAN SACHS INTERNATIONAL
XS1267221841 ,  KINGDOM OF THE NETHERLANDS ,  GOLDMAN SACHS INTERNATIONAL
XS1266266813 ,  KINGDOM OF THE NETHERLANDS ,  BARCLA

In [12]:
data.evaluate([8],labels,printerr=True, match_ok=100) # removing line and spliting on sentence.

XS1051380837 ,  None ,  DEUTSCHE BANK AG
XS1324899985 ,  None ,  HSBC BANK PLC
XS1336933194 ,  None ,  HSBC BANK PLC
XS1183228789 ,  NATIONAL AUSTRALIA BANK LTD ,  CREDIT SUISSE INTERNATIONAL
XS1336567315 ,  None ,  HSBC BANK PLC
XS1332081352 ,  None ,  HSBC BANK PLC
XS1051372495 ,  None ,  DEUTSCHE BANK AG
XS0461332347 ,  None ,  DEUTSCHE BANK AG
XS1330994689 ,  None ,  ROYAL BANK OF CANADA
XS1051364633 ,  None ,  DEUTSCHE BANK AG
XS1254327510 ,  STANDARD CHARTERED PLC ,  CREDIT SUISSE INTERNATIONAL
XS1340707659 ,  None ,  HSBC BANK PLC
XS1340128625 ,  None ,  HSBC BANK PLC
XS1349641206 ,  None ,  HSBC BANK PLC
XS1051365010 ,  None ,  DEUTSCHE BANK AG
XS1051340963 ,  None ,  DEUTSCHE BANK AG
XS1051371414 ,  None ,  DEUTSCHE BANK AG
XS1221134189 ,  None ,  DEUTSCHE BANK AG
XS1313797141 ,  None ,  BNP PARIBAS ARBITRAGE ISSUANCE
XS1324210563 ,  None ,  HSBC BANK PLC
XS1354473230 ,  None ,  HSBC BANK PLC
XS1330444420 ,  None ,  HSBC BANK PLC
XS1323908944 ,  None ,  HSBC BANK PLC
DE000UZ16

In [11]:
from code.Matching import fuzzCompany2
fcomp = fuzzCompany2.from_file('fc_data.pkl')
a=u'linked to national australia bank ltd ord due                              2016 (the securities)          d'
#print fcomp.get_Ps(a)
#for item in fcomp.to_numberset(a)[0]:
#    print item
print fcomp.closerMatch(a)

(0.3333333333333333, 'NATIONAL AUSTRALIA BANK LTD')


By direct vectorization:
    ---------------------

In [40]:
from code.Matching import fuzzCompany2
import os
from code import Text_reader as tr
from code.Labels import label
from code.Text_reader import htmlDoc
import copy 
import re
import numpy as np

data0='../data_0'
data1='../data_1'
labels=label.from_cvs(data0)
data=copy.deepcopy(labels)
data.empty()

fcomp = fuzzCompany2.from_file('fc_data2.pkl')

['fileId', 'NOMINAL.CURR', 'MIN.TRAD.AMT', 'MULT.TRAD.AMT', 'ZERO.COUPN.FLAG', 'SEC.SUB.ID', 'FUNG.FL', 'OpCurrency', 'issuerName', 'issuerCity', 'issuerCountry', 'guarantorName', 'guarantorCity', 'guarantorCountry', 'City.Id', 'City.Name']


In [45]:
def get_issuer(txt):
    txtc = re.sub("[^a-zA-Z0-9\s]",           # The pattern to search for
                      "",                   # The pattern to replace it with
                      txt)
    txtc=txtc.replace('\n',' ').replace('\r','').lower().split()
    v=[-1]
    tmp=[]
    for x in xrange(0,len(txtc)):
        if txtc[x] in fcomp.Lookup:
            tmp.append(fcomp.Lookup[txtc[x]])
        else:
            if len(tmp) > 0:
                v.append(tmp)
                tmp=[]
            if isinstance(v[-1],int):
                v[-1] -= 1
            else:
                v.append(-1)

        if txtc[x] in ['issue','issued']:  
            if not isinstance(v[-1],int):
                v.append(0)
            v.append('-')

        if txtc[x] in ['issuer','issued']:
            v.append('+')
            v.append(0)

        if len(txtc)> x+1 and txtc[x] == 'linked' and txtc[x+1] == 'to':
            v.append('*')

        if len(txtc)> x+1 and txtc[x] == 'share' and txtc[x+1] == 'issuer':
            v.append('*')

    group=[]
    x=0
    while x<len(v):
        if v[x]=='*' and x+2 < len(v):
            x+=2
            continue
        if v[x]=='-' and x > 2:
            group.append([abs(v[x-1]),v[x-2]])
        if v[x]=='+' and x+2 < len(v):
            group.append([abs(v[x+1]),v[x+2]]) 
        x+=1

    r=[]

    #x=0
    for item in group:
        subset = fcomp.get_subsets(item[1])
        ps = fcomp.get_Ps_vect(subset)
        idx = np.argmax(ps)
        #print x,fcomp.values[idx]
        #x+=1
        r.append((ps[idx]/float(item[0]+1), fcomp.values[idx]))

    r = sorted(r,key=lambda x:x[0],reverse=True)
    if len(r)>0:
        return r[0][1]
    else:
        return None


In [46]:
folder='../data_0/html'
files=tr.get_files(folder)
filesTxT=tr.get_files('../data_0/txt')

for isin in data: # for each isin in the data object created before
#for isin in ['XS1183228789']:
    f=data[isin][0] # get the file_ids
    issuer=None
    for item in f: # for each of these files
        # get the txt.
        txt=tr.clean_text(tr.get_text(filesTxT[item],raw=True))
        issuer=get_issuer(txt)
        if issuer is None:
            txt=tr.clean_text(tr.get_text(files[item]))
            issuer=get_issuer(txt)
            
        if issuer is not None:
            break
    
    print isin, issuer
    if issuer is not None:
        data[isin][8]=issuer

XS1240015799 CREDIT SUISSE INTERNATIONAL
XS1293137904 BNP PARIBAS ARBITRAGE ISSUANCE BV
XS1313796846 BNP PARIBAS ARBITRAGE ISSUANCE BV
XS1313802677 BNP PARIBAS ARBITRAGE ISSUANCE BV
XS1313800895 BNP PARIBAS ARBITRAGE ISSUANCE BV
XS1267246327 GOLDMAN SACHS INTERNATIONAL
XS1275964507 DEUTSCHE BANK
XS1329979261 THE HONGKONG AND SHANGHAI BANKING CORPORATION LIMITED
XS1312160507 ROYAL BANK OF CANADA
XS1253932765 CREDIT SUISSE INTERNATIONAL
XS1267246244 GOLDMAN SACHS INTERNATIONAL
XS1293817877 THE HONGKONG AND SHANGHAI BANKING CORPORATION LIMITED
XS1051370440 TOTAL SYSTEM SERVICES INC
XS1267221841 GOLDMAN SACHS INTERNATIONAL
XS1266266813 BARCLAYS BANK PLC
XS1313804459 BNP PARIBAS ARBITRAGE ISSUANCE BV
XS1332626016 ROYAL BANK OF CANADA
XS1338871996 SMBC NIKKO SECURITIES INC.
XS1051380837 TOTAL SYSTEM SERVICES INC
XS1313800119 BNP PARIBAS ARBITRAGE ISSUANCE BV
XS1293807720 THE HONGKONG AND SHANGHAI BANKING CORPORATION LIMITED
XS1326568430 ROYAL BANK OF CANADA
XS1222904416 BNP PARIBAS ARBITRAGE

In [47]:
data.evaluate([8],labels,printerr=True, match_ok=100) # removing line and spliting on sentence.

XS1051380837 ,  TOTAL SYSTEM SERVICES INC ,  DEUTSCHE BANK AG
XS1051370010 ,  AMERICAN INTERNATIONAL GROUP, INC. ,  DEUTSCHE BANK AG
XS1051375910 ,  AMERICAN INTERNATIONAL GROUP, INC. ,  DEUTSCHE BANK AG
XS1339631548 ,  BANK OF AMERICA CORPORATION ,  ROYAL BANK OF CANADA
XS1326563043 ,  CHINA CONSTRUCTION BANK CORPORATION ,  ROYAL BANK OF CANADA
XS1051372495 ,  TOTAL SYSTEM SERVICES INC ,  DEUTSCHE BANK AG
XS1330994689 ,  SG ISSUER ,  ROYAL BANK OF CANADA
XS1051362694 ,  TOTAL SYSTEM SERVICES INC ,  DEUTSCHE BANK AG
XS1051364633 ,  TOTAL SYSTEM SERVICES INC ,  DEUTSCHE BANK AG
XS1051370879 ,  TOTAL SYSTEM SERVICES INC ,  DEUTSCHE BANK AG
XS1051374350 ,  TOTAL SYSTEM SERVICES INC ,  DEUTSCHE BANK AG
XS1190839586 ,  J.P. MORGAN SECURITIES PLC ,  J.P. MORGAN STRUCTURED PRODUCT
XS1326555619 ,  AUSTRALIA AND NEW ZEALAND BANKING GROUP LIMITED ,  ROYAL BANK OF CANADA
XS1051374277 ,  TOTAL SYSTEM SERVICES INC ,  DEUTSCHE BANK AG
XS1051338041 ,  AMERICAN INTERNATIONAL GROUP, INC. ,  DEUTSCHE BA

In [31]:
f = filesTxT[item]

In [32]:
txt=tr.get_text(filesTxT[item],raw=True)

In [33]:
txtc = re.sub("[^a-zA-Z0-9\s]",           # The pattern to search for
                      "",                   # The pattern to replace it with
                      txt)
txtc=txtc.replace('\n',' ').replace('\r','').lower().split()

In [34]:
v=[-1]
tmp=[]
for x in xrange(0,len(txtc)):
    if txtc[x] in fcomp.Lookup:
        tmp.append(fcomp.Lookup[txtc[x]])
    else:
        if len(tmp) > 0:
            v.append(tmp)
            tmp=[]
        if isinstance(v[-1],int):
            v[-1] -= 1
        else:
            v.append(-1)
            
    if txtc[x] in ['issue','issued']:  
        if not isinstance(v[-1],int):
            v.append(0)
        v.append('-')
        
    if txtc[x] in ['issuer','issued']:
        v.append('+')
        v.append(0)
        
    if len(txtc)> x+1 and txtc[x] == 'linked' and txtc[x+1] == 'to':
        v.append('*')
     
    if len(txtc)> x+1 and txtc[x] == 'share' and txtc[x+1] == 'issuer':
        v.append('*')
        
        
group=[]
x=0
while x<len(v):
    if v[x]=='*' and x+2 < len(v):
        x+=2
        continue
    if v[x]=='-' and x > 2:
        group.append([abs(v[x-1]),v[x-2]])
    if v[x]=='+' and x+2 < len(v):
        group.append([abs(v[x+1]),v[x+2]]) 
    x+=1


In [35]:
#[399, 5, 155, 11]],
fcomp.vocab[399]

'issuer'

In [36]:
for x in xrange(0, len(v)):
    if isinstance(v[x],list) and 281 in v[x]:
        print v[x-5:x+5]

In [37]:
import numpy as np

r=[]

#x=0
for item in group:
    subset = fcomp.get_subsets(item[1])
    ps = fcomp.get_Ps_vect(subset)
    idx = np.argmax(ps)
    #print x,fcomp.values[idx]
    #x+=1
    r.append((ps[idx]/float(item[0]+1), fcomp.values[idx]))
    
r = sorted(r,key=lambda x:x[0],reverse=True)
print r[0]

(0.5, 'CREDIT SUISSE INTERNATIONAL')


44

In [17]:
item=[1016, 1017, 41, 355, 281, 548, 34, 501]
subset = fcomp.get_subsets(item)
ps = fcomp.get_Ps_vect(subset)
idx = np.argmax(ps)
print ps[idx], fcomp.values[idx]

0.375 CREDIT SUISSE INTERNATIONAL


In [33]:
fcomp.vocab[1016]

'stock'

In [39]:
#txt

In [34]:
w='stock exchange 1 Issuer Credit Suisse International'
fcomp.closerMatch(w)

(0.42857142857142855, 'CREDIT SUISSE INTERNATIONAL')

In [35]:
fcomp.closerMatch('stock exchange 1 Issuer Credit Suisse International')

(0.42857142857142855, 'CREDIT SUISSE INTERNATIONAL')

In [18]:
folder='../Newdata/train/html'
files=tr.get_files(folder)